In [1]:
from dwave.system.composites import EmbeddingComposite
from dwave.system.samplers import DWaveSampler
import numpy as np
from pyqubo import Spin
import dimod

In [13]:
#Función para entrar el problema SAT (en CNF) y tener una lista con las cláusulas
def pedir_problema_SAT():
    clauses = []
    ruta = "s2v100c1200-1.cnf"
    with open(ruta, "r") as entrada:
        clauses = entrada.readlines()
    return clauses

In [14]:
# Pedir el problema SAT
clauses = pedir_problema_SAT()

In [15]:
#Identificar el x_n del SAT introducido y crear las h y J
b = int(list(clauses[2].split(" "))[2])
J = {(i, j): 0 for i in range(b) for j in range(i+1, b)}
h = {i: 0 for i in range(b)}

In [16]:
print(b)

100


In [17]:
#Determinamos las v={-1,0,1} (que equivale a si la variable está negada o si no está en el problema) y calculamos h y J
for i in range(3,len(clauses)): #Buscle para cada cláusula
    #Convertimos cláusula en listado de integgers (que son las variables del SAT)
    variables = list(map(int, clauses[i].split(" ")[:-1]))
    #Creamos una lista con el signo que tiene cada variable (la negación) y pasamos todas las variables de sat a positivo (para no tener problemas al identificar las variables)
    sign=[0]*2
    for j in range(2):
        if variables[j]>0:
            h[variables[j]-1]-=1 #Restamos en h[variables[j]-1] porque la variable del sat que es 1, en la lista corresponde al 0
            sign[j]=1
        if variables[j]<0:
            variables[j]=-variables[j] #Todas las variables positivas
            h[variables[j]-1]+=1
            sign[j]=-1
    if variables[0]<variables[1]:
        J[variables[0]-1,variables[1]-1]+=(sign[0]*sign[1])
    else:
        J[variables[1]-1,variables[0]-1]+=(sign[0]*sign[1])

print(h) #Solo con la h ya nos hacemos una idea

{0: -1, 1: -4, 2: -4, 3: 0, 4: -3, 5: -4, 6: -3, 7: -1, 8: 4, 9: 9, 10: -3, 11: 7, 12: 2, 13: -8, 14: 2, 15: -2, 16: -4, 17: -7, 18: -6, 19: 4, 20: -1, 21: -2, 22: 0, 23: 4, 24: 9, 25: 7, 26: -5, 27: -2, 28: -3, 29: -3, 30: -2, 31: -4, 32: 5, 33: -2, 34: -9, 35: -6, 36: 4, 37: -4, 38: -2, 39: 2, 40: 8, 41: 0, 42: -4, 43: 3, 44: -2, 45: 2, 46: 1, 47: 2, 48: -2, 49: 7, 50: -3, 51: 2, 52: -4, 53: -1, 54: 4, 55: -4, 56: 6, 57: -5, 58: 0, 59: 10, 60: 2, 61: -4, 62: -5, 63: -9, 64: 0, 65: 8, 66: -1, 67: -5, 68: 8, 69: 1, 70: -6, 71: -5, 72: 8, 73: -1, 74: 6, 75: 0, 76: -5, 77: 1, 78: 6, 79: -2, 80: 7, 81: 4, 82: -5, 83: 0, 84: 0, 85: -5, 86: -2, 87: 0, 88: -12, 89: -7, 90: 3, 91: -4, 92: 1, 93: 6, 94: -3, 95: -1, 96: -8, 97: -5, 98: 5, 99: -3}


In [18]:
secure_q = input("Do you really want to use the QPU? (y/n)")
if secure_q == 'y' or secure_q == 'Y':
    sampler = EmbeddingComposite(DWaveSampler(token='DEV-291d80af600d6eb433a8019c579070ba37436e9a'))
    sample_set = sampler.sample_ising(h, J, num_reads=500, return_embedding=True)
    print("Using DWaveSampler()")
    print(sample_set)

Do you really want to use the QPU? (y/n) y


Using DWaveSampler()
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 ... 99 energy num_oc. ...
28  -1 +1 -1 -1 +1 +1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -512.0       1 ...
94  -1 +1 -1 -1 +1 +1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -512.0       1 ...
388 -1 +1 +1 -1 +1 +1 +1 -1 +1 -1 +1 -1 -1 +1 -1 +1 ... +1 -512.0       1 ...
0   -1 +1 +1 -1 +1 -1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -508.0       1 ...
3   -1 +1 +1 +1 +1 -1 +1 -1 +1 -1 -1 -1 +1 +1 -1 +1 ... +1 -508.0       1 ...
27  -1 +1 +1 -1 +1 -1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -508.0       1 ...
2   -1 +1 -1 -1 +1 +1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -504.0       1 ...
5   -1 +1 +1 -1 +1 +1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -504.0       1 ...
141 +1 +1 +1 -1 +1 +1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -504.0       1 ...
4   -1 +1 +1 -1 +1 +1 +1 -1 +1 -1 +1 -1 +1 +1 -1 +1 ... +1 -500.0       1 ...
13  -1 +1 +1 -1 +1 -1 +1 -1 +1 -1 -1 -1 +1 +1 -1 +1 ... +1 -500.0       1 ...
58  -1 +1 +1 -1 +1 -1 +1 -1 +1 -1 -1 -1 +1 

In [19]:
#Tiempo que tarda cada acción en el QPU (en microsegundos)
print(sample_set.info)

{'timing': {'qpu_sampling_time': 122280.0, 'qpu_anneal_time_per_sample': 20.0, 'qpu_readout_time_per_sample': 204.02, 'qpu_access_time': 138046.37, 'qpu_access_overhead_time': 1784.63, 'qpu_programming_time': 15766.37, 'qpu_delay_time_per_sample': 20.54, 'total_post_processing_time': 1899.0, 'post_processing_overhead_time': 1899.0}, 'problem_id': 'a0ef3e26-3001-4880-91c0-72ca3e1afdf3', 'embedding_context': {'embedding': {1: (154, 156, 155, 4170, 4171, 153, 4174, 4173, 152, 4172), 0: (1217, 1218, 1219, 3857, 3859, 3856, 3860, 3861, 3855, 3858, 1280), 2: (857, 858, 859, 860, 861, 4306, 862, 4305, 4308, 863, 4307), 3: (602, 603, 604, 605, 606, 607, 608, 609, 4711, 4712), 4: (287, 288, 289, 290, 291, 263, 293, 4591, 292, 4593, 4592), 5: (1037, 1038, 4398, 4396, 4399, 1042, 4175, 1041, 4220, 1040, 4397, 4190, 4400, 1039), 6: (482, 484, 485, 486, 487, 488, 483, 4621, 354, 489, 490, 4846), 7: (872, 873, 874, 875, 876, 877, 4352, 4353, 4351, 4354), 8: (753, 755, 756, 758, 4473, 757, 4470, 4471

In [20]:
#Tiempo usado para resolver la QPU (teniendo en cuenta la configuración y demás
qpu_access_time = sample_set.info['timing']['qpu_access_time']
#Tiempo usado por la QPU para resolver el problema de forma activa
qpu_sampling_time = sample_set.info['timing']['qpu_sampling_time']
print(qpu_access_time, qpu_sampling_time)

138046.37 122280.0


In [21]:
#Copiamos resultados en un archivo aparte con un formato específico
def convert_to_format(result):
    return 'v ' + ''.join('1' if result[v] == 1 else '0' for v in result)

#Abre el archivo para escribir las soluciones
cnf_file = "s2v100c1200-1.cnf"
output_file = f"{cnf_file}.dwave.sol"
with open(output_file, 'w') as output_file:
    #Escribimos el tiempo que ha tardado en resolver el problema
    output_file.write(f"Total time the QPU has spent processing the problem (qpu_access_time): {qpu_access_time}\n"
    f"Total time the QPU has spent on active resolution (qpu_sampling_time): {qpu_sampling_time}\n")
    
    #Recorre los resultados y escribe cada uno en el archivo
    for result in sample_set:
        formatted_result = convert_to_format(result)
        output_file.write(formatted_result + '\n')

print(f'Soluciones guardadas en {output_file}')

Soluciones guardadas en <_io.TextIOWrapper name='s2v100c1200-1.cnf.dwave.sol' mode='w' encoding='UTF-8'>


In [58]:
# Obtener el embedding del resultado
embedding = sample_set.info['embedding']

# Visualizar el embedding
sampler.child.embedding.draw_embedding(embedding)

KeyError: 'embedding'

In [7]:
## Solving with exact solver

In [63]:
bqm = dimod.BinaryQuadraticModel.from_ising(h, J)
Solver=dimod.ExactSolver()
sampleset = Solver.sample(bqm)
print(sampleset)

ValueError: Maximum allowed dimension exceeded